In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

#from pandas_profiling import ProfileReport
! pip install pandas-profiling[notebook]
print('Setup complete')


# 1. Loading data from input directory...

In [2]:
# Path of the file to read
#submission_filepath = "../input/tabular-playground-series-apr-2021/sample_submission.csv"
#test_filepath = "../input/tabular-playground-series-apr-2021/test.csv"
train_filepath = "../input/tabular-playground-series-apr-2021/test.csv"

# Read the file into a variables 
#submission_data = pd.read_csv(submission_filepath, index_col="PassengerId", parse_dates=True)
#test_data = pd.read_csv(test_filepath, index_col="PassengerId", parse_dates=True)
train_data = pd.read_csv(train_filepath, index_col="PassengerId", parse_dates=True)

# 2. Check data quality

In [3]:
train_data.describe()

2.1 Missing values

In [52]:
missing_values = sns.displot(
    data=train_data.isna().melt(value_name="missing"),
    y="variable",
    hue="missing"
)

2.2 Unique values

In [50]:
plt.figure(figsize=(14,8))
nu = train_data.nunique().reset_index()
nu.columns = ['columns','nunique']
sns.barplot(x='columns', y='nunique', data=nu)

My_notes: 
There are some warrings in data quality in file train.csv
1. there are empty cells in columns "Age", "Fare", "Ticket", "Cabin", "Embarked"
2. there are enough namesakes
3. all values are unique in column "PassengerId"

# 3. Data Analyzing

3.1 Dependence between variables

In [46]:
plt.figure(figsize=(14,8))
corr = train_data.corr()
heatmap = sns.heatmap(corr, annot = True)


My_note:
1. **Inverse correlation between Pclass and Fare.** Its looks true, because 1st class tickets are more expensive then 3st class tickets
2. **Possitive correlation between Parch and Fare.** Its looks also true, because parents with there childen usually stay in one cabin and cabin for 3 person is more expensive then single cabin 

In [56]:
sns.displot(train_data, y="Pclass", hue="Sex")
sns.displot(train_data, y="Age", hue="Sex")

In [38]:
from pandas_profiling import ProfileReport

Note: 

In [9]:
profile = ProfileReport(train_data, title="Train_data Profiling Report")
profile.to_widgets()